In [2]:
import pandas as pd
import math, datetime
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import style
import pickle
import re
from sklearn.cluster import KMeans
style.use('ggplot')
%matplotlib inline

In [3]:
df = pd.read_csv('testing.csv')
df

,Unnamed: 0,ID,Name,Page Load,Feed Impression,Click Grid,Feed Impression Rate,Feed Conversion Rate
0,1,832,site1.ru - fashion,56907,42387,297,74.5%,0.7%
1,2,685,site2.ru,4694866,3306119,9859,70.4%,0.3%
2,3,206,site4.ru - beauty,22114,15370,112,69.5%,0.7%
3,4,540,trip site,278,175,3,62.9%,1.7%
4,5,205,moviesite,1736,1013,9,58.4%,0.9%
5,6,517,sportsite,42232,23260,201,55.1%,0.9%
6,7,740,beauty site tab,644,347,57,53.9%,16.4%
7,8,518,site5,24978,13217,169,52.9%,1.3%
8,9,115,site6,1740346,866389,2026,49.8%,0.2%
9,10,133,site7,58691,28145,317,48.0%,1.1%


In [4]:
# Cleaning The Data
df['Feed Conversion Rate'] = df['Feed Conversion Rate'].apply(lambda x: float(x.replace('%', '')))

In [5]:
df

,Unnamed: 0,ID,Name,Page Load,Feed Impression,Click Grid,Feed Impression Rate,Feed Conversion Rate
0,1,832,site1.ru - fashion,56907,42387,297,74.5%,0.7
1,2,685,site2.ru,4694866,3306119,9859,70.4%,0.3
2,3,206,site4.ru - beauty,22114,15370,112,69.5%,0.7
3,4,540,trip site,278,175,3,62.9%,1.7
4,5,205,moviesite,1736,1013,9,58.4%,0.9
5,6,517,sportsite,42232,23260,201,55.1%,0.9
6,7,740,beauty site tab,644,347,57,53.9%,16.4
7,8,518,site5,24978,13217,169,52.9%,1.3
8,9,115,site6,1740346,866389,2026,49.8%,0.2
9,10,133,site7,58691,28145,317,48.0%,1.1


In [6]:
# Cleaning The Data
df['Feed Impression Rate'] = df['Feed Impression Rate'].apply(lambda x: float(x.replace('%', '')))

In [7]:
df

,Unnamed: 0,ID,Name,Page Load,Feed Impression,Click Grid,Feed Impression Rate,Feed Conversion Rate
0,1,832,site1.ru - fashion,56907,42387,297,74.5,0.7
1,2,685,site2.ru,4694866,3306119,9859,70.4,0.3
2,3,206,site4.ru - beauty,22114,15370,112,69.5,0.7
3,4,540,trip site,278,175,3,62.9,1.7
4,5,205,moviesite,1736,1013,9,58.4,0.9
5,6,517,sportsite,42232,23260,201,55.1,0.9
6,7,740,beauty site tab,644,347,57,53.9,16.4
7,8,518,site5,24978,13217,169,52.9,1.3
8,9,115,site6,1740346,866389,2026,49.8,0.2
9,10,133,site7,58691,28145,317,48.0,1.1


In [8]:
X = df[['Feed Impression', 'Click Grid','Page Load']]
y = df.pop('Feed Conversion Rate')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.33, random_state=42)
 
# Standardise The Data 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns, index = X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns, index = X_test.index)

In [10]:
# Import different Algorithms to see differences between their predictions
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
 
Linear_regression = LinearRegression()
Linear_regression.fit(X_train, y_train)
Linear_regression.score(X_train, y_train)
Linear_regression.score(X_test, y_test)

-40430761.44203356

In [11]:
RandomForestRegressorModel = RandomForestRegressor(n_estimators=50)
RandomForestRegressorModel.fit(X_train, y_train)
prediction_score = RandomForestRegressorModel.score(X_train, y_train)
test_score = RandomForestRegressorModel.score(X_test, y_test)

In [29]:
print(prediction_score,test_score)

0.6965787736858777 -9.345673282442746


In [33]:
#Creating a function that combines predict For X_train, X_test and also contains the index numbers so that we can match it back up with the original dataframe.
X_list = [X_train, X_test]
y_list = y_train, y_test
 
RandomForestRegressorModel = RandomForestRegressor(n_estimators=50)
 
data_dict = {
    'Indexes': [],
    'Predictions': []    
}
 
for x, y in zip(X_list, y_list):
    
    indexes = list(x.index)
    RandomForestRegressorModel.fit(X_train, y_train)
    predictions = RandomForestRegressorModel.predict(x)
    
    data_dict['Indexes'].extend(indexes)
    data_dict['Predictions'].extend(predictions)

In [14]:
data_dict

{'Indexes': [7, 2, 9, 4, 3, 6, 8, 1, 5, 0],
 'Predictions': [1.779999999999999,
  1.742000000000001,
  1.1040000000000005,
  3.2460000000000013,
  5.179999999999996,
  10.685999999999993,
  1.1360000000000003,
  1.1360000000000003,
  1.1280000000000003,
  1.1360000000000003]}

In [15]:
merged_X = pd.concat([X_train, X_test])
merged_Y = pd.concat([y_train, y_test])
merged_df = pd.concat([merged_X, merged_Y], axis = 1)
merged_df.head(10)

,Feed Impression,Click Grid,Page Load,Feed Conversion Rate
7,0.339875,0.532456,0.331607,1.3
2,0.548599,0.007672,0.194056,0.7
9,1.787078,1.895052,1.950766,1.1
4,-0.843249,-0.940621,-0.784653,0.9
3,-0.924489,-0.995862,-0.854677,1.7
6,-0.907814,-0.498698,-0.837099,16.4
8,83.051115,17.629358,82.716840,0.2
1,319.572114,89.745693,224.615732,0.3
5,1.313499,0.827071,1.160278,0.9
0,3.167777,1.710918,1.865085,0.7


In [16]:
print(merged_df)

   Feed Impression  Click Grid   Page Load  Feed Conversion Rate
7         0.339875    0.532456    0.331607                   1.3
2         0.548599    0.007672    0.194056                   0.7
9         1.787078    1.895052    1.950766                   1.1
4        -0.843249   -0.940621   -0.784653                   0.9
3        -0.924489   -0.995862   -0.854677                   1.7
6        -0.907814   -0.498698   -0.837099                  16.4
8        83.051115   17.629358   82.716840                   0.2
1       319.572114   89.745693  224.615732                   0.3
5         1.313499    0.827071    1.160278                   0.9
0         3.167777    1.710918    1.865085                   0.7


In [17]:
predicted_data = pd.DataFrame(data_dict)
predicted_data.set_index(predicted_data['Indexes'], inplace = True)
predicted_data.drop(columns=['Indexes'], inplace = True)
predicted_data.rename(columns={"Predictions": "Predicted_CTR_%"}, inplace = True)

In [18]:
predicted_data.head(6)

,Predicted_CTR_%
Indexes,
7,1.780
2,1.742
9,1.104
4,3.246
3,5.180
6,10.686


In [19]:
final_df = pd.concat([merged_df, predicted_data], axis=1)

In [20]:
final_df

,Feed Impression,Click Grid,Page Load,Feed Conversion Rate,Predicted_CTR_%
7,0.339875,0.532456,0.331607,1.3,1.780
2,0.548599,0.007672,0.194056,0.7,1.742
9,1.787078,1.895052,1.950766,1.1,1.104
4,-0.843249,-0.940621,-0.784653,0.9,3.246
3,-0.924489,-0.995862,-0.854677,1.7,5.180
6,-0.907814,-0.498698,-0.837099,16.4,10.686
8,83.051115,17.629358,82.716840,0.2,1.136
1,319.572114,89.745693,224.615732,0.3,1.136
5,1.313499,0.827071,1.160278,0.9,1.128
0,3.167777,1.710918,1.865085,0.7,1.136


In [21]:
final_df['CTR_Difference'] = final_df['Predicted_CTR_%'] - final_df['Feed Conversion Rate']

In [22]:
final_df = final_df[final_df['Feed Conversion Rate'] != 0.00]

In [23]:
final_df.sort_values(by='CTR_Difference', ascending=False)

,Feed Impression,Click Grid,Page Load,Feed Conversion Rate,Predicted_CTR_%,CTR_Difference
3,-0.924489,-0.995862,-0.854677,1.7,5.180,3.480
4,-0.843249,-0.940621,-0.784653,0.9,3.246,2.346
2,0.548599,0.007672,0.194056,0.7,1.742,1.042
8,83.051115,17.629358,82.716840,0.2,1.136,0.936
1,319.572114,89.745693,224.615732,0.3,1.136,0.836
7,0.339875,0.532456,0.331607,1.3,1.780,0.480
0,3.167777,1.710918,1.865085,0.7,1.136,0.436
5,1.313499,0.827071,1.160278,0.9,1.128,0.228
9,1.787078,1.895052,1.950766,1.1,1.104,0.004
6,-0.907814,-0.498698,-0.837099,16.4,10.686,-5.714


In [24]:
final_df = final_df[['Feed Conversion Rate', 'Predicted_CTR_%', 'CTR_Difference']]

In [25]:
original_data = df[['Page Load', 'Click Grid', 'Feed Impression', 'Name']]

In [26]:
results = pd.merge(original_data, final_df, left_index=True, right_index=True)

In [27]:
results.to_csv('results.csv')

In [28]:
print(results)

   Page Load  Click Grid  Feed Impression                  Name  \
0      56907         297            42387  site1.ru  -  fashion   
1    4694866        9859          3306119              site2.ru   
2      22114         112            15370   site4.ru  -  beauty   
3        278           3              175             trip site   
4       1736           9             1013             moviesite   
5      42232         201            23260             sportsite   
6        644          57              347      beauty site  tab   
7      24978         169            13217                 site5   
8    1740346        2026           866389                 site6   
9      58691         317            28145                 site7   

   Feed Conversion Rate  Predicted_CTR_%  CTR_Difference  
0                   0.7            1.136           0.436  
1                   0.3            1.136           0.836  
2                   0.7            1.742           1.042  
3                   1.7   